In [1]:
from tensorflow.io import read_file
from tensorflow.image import decode_jpeg, convert_image_dtype, resize
from tensorflow import reshape
import tensorflow as tf
import json
import numpy as np

In [2]:
def read_json(json_file_path):

    with open(json_file_path, 'r') as f:
        data = json.load(f)
    return data

In [4]:
def get_image_data(data_dict, base_img_dir = 'Data/ship/'):
    '''
        Implement the function to get the image path and corresponding camera transformation matrix.
    '''

    img_paths = []
    img_cam_transform_matrices = []

    all_frames = data_dict["frames"]


    return

In [2]:
class getImage():
	def __init__(self, imageWidth, imageHeight):

		self.imageWidth = imageWidth
		self.imageHeight = imageHeight

	def __call__(self, imagePath):

		image = read_file(imagePath)
		image = decode_jpeg(image, 3)
		image = convert_image_dtype(image, dtype=tf.float32)
		image = resize(image, (self.imageWidth, self.imageHeight))
		image = reshape(image, (self.imageWidth, self.imageHeight, 3))

		return image

In [ ]:
def get_rays_for_image_matrix(width, height, camera2world_matrix, focal_length):

    r_mat = camera2world_matrix[:3, :3]
    o_vec = camera2world_matrix[:3, -1:]

    i, j =  np.meshgrid(
            np.arange(height, dtype=np.float32),
            np.arange(width, dtype=np.float32),
                        indexing='ij')

    vector_x = j - width*0.5
    vector_y = - (i - height*0.5)
    vector_z = - np.ones_like(vector_x)*focal_length

    dirs = np.stack((vector_x, vector_y, vector_z), axis=2)
    ray_dirs = np.sum(r_mat * dirs[:, :, None, :], axis=3)

    origins = np.ones_like(ray_dirs)*o_vec.squeeze()

    return ray_dirs, origins

In [1]:
def get_sampled_ray_points(ray_dirs, near=1, far=10, num_samples=20):

    bins = np.linspace(near, far, num_samples)
    ran_samples = np.random.uniform(size=(list(ray_dirs.shape[:-1])+[num_samples]))  # generate random samples for each point of the input matrix
    scaled_ran_samples = ran_samples*(far-near)/num_samples
    ray_samples = scaled_ran_samples + np.broadcast_to(bins, shape=(list(ray_dirs.shape[:-1])+[num_samples]))

    return ray_samples

In [ ]:
import tensorflow as tf
class NeRFModel(tf.Module):

    def __init__(self, name=None):
        super().__init__(name)

        





    pass